# 🙤 **PARTIE Ⅰ** 🙧 
# Peuplement d'un graphe de connaissance sur l'œuvre photographique d'[Eugène Atget](https://fr.wikipedia.org/wiki/Eug%C3%A8ne_Atget) à partir des portails de données de la bibliothèque nationale de France

<div style="display: flex; justify-content: center; align-items: center; gap: 20px; flex-wrap: wrap;">
    <img src="assets/atget_1.jpg" alt="New Monico, Atget 1925" width="700" /> 
    <img src="assets/atget_2.jpg" alt="Portrait d'Eugène Atget vers 1890" width="282" />
</div>

🙑 Eugène Atget (1857‑1927) est un photographe français, pionnier de la photographie documentaire, actif au tournant du XX<sup>e</sup> siècle. Il a construit une œuvre photographique monumentale qui documente avec rigueur le Paris de la Belle Époque, de la Zone et ses environs, avant et pendant les profondes transformations urbaines du début du XXᵉ siècle. Ses images décrivent les rues, les façades, les boutiques, les cours, les jardins et les scènes de la vie quotidienne avec une précision visuelle remarquable, cherchant moins à créer une esthétique artificielle qu’à enregistrer minutieusement la réalité urbaine. Atget n’aspirait pas à être vu comme un artiste au sens traditionnel : il envisageait la photographie d’abord comme un outil documentaire permettant de fixer des éléments de la ville menacés de disparition. Cependant, au fil du temps Atget évolue de la simple documentation architecturale (1890-1905) et adopte un regard plus attentif aux traces de vie humaine et sociale, photographiant artisans, travailleurs des rues et habitants de la zone. L’accumulation de ces vues confère à son œuvre une puissance métaphysique et poétique qui a séduit bien au‑delà de son intention initiale, influençant notamment les surréalistes et des générations de photographes modernes. Dans sa diversité, l’œuvre d’Atget se situe à l’intersection du document social et historique et d’une forme naissante de lecture critique de l’espace urbain, interrogeant la visibilité des corps et des constructions dans le paysage citadin.
Aujourd’hui, la BnF conserve plus de 4 500 épreuves positives d’Atget, réparties en séries thématiques élaborées par lui‑même ou par la bibliothèque, ainsi que plusieurs albums qu’il a constitués. Cette immense archive, complétée par d’autres fonds publics et privés, fait d’Atget une figure centrale de la photographie documentaire et un témoignage unique du vieux Paris, oscillant entre la constatation archivistique et l’évocation sensible du temps qui passe. 🙗

<small>Source : Rizov, V. (2021). Eugène Atget and documentary photography of the city. Theory, Culture & Society, 38(3), 141-163. Pour en apprendre plus sur Eugène Atget et son oeuvre : https://expositions.bnf.fr/atget/arret/01.htm et https://gallica.bnf.fr/accueil/fr/html/sur-les-traces-du-paris-datget

Photos : à gauche le "New Monico", 1925. À droite : portrait photographique d'Eugène Atget vers 1890, auteur inconnu.
</small>

## 🙤 Objectifs

Cette première partie guide la construction d'une base de données locale indexant les photographies d'Eugène Atget archivées, inventoriées et numérisées par la BnF. Les métadonnées utiles sont exposées sur différents points d'accès de la BnF : [Gallica]() pour les numérisations, [le catalogue général de la bibliothèque]() pour les notices bibliographiques, [data.bnf.fr]() pour les métadonnées publiques en RDF.

Nous allons construire cette base de données sous la forme d'un **graphe de connaissances RDF** en collectant les informations sur les photographies de Atget à partir des **APIs** de la BnF. Ces donnes nous serviront plus tard pour **géolocaliser** réaliser un **explorateur cartographique interactif** du fond Atget de la BnF.

Le modèle du graphe de connaissances est pertinent  pour trois raisons : 
1.  les données sont **déjà partiellement disponibles en triplets RDF** ;
2.  les **métadonnées très imbriquées** de la BnF s'adaptent très bien à une représentation en graphe, mais très mal à de "simples" tableaux  ;
3.  un graphe de connaissance est **facile à enrichir** de nouvelles informations

Nous allons voir :
- les différents points d'accès aux métadonnées de la BnF et les modes de récupération liés : API SRU de Gallica, SPARQL endpoint de data.bnf.fr
- comment exécuter des requêtes SPARQL distantes en Python avec *SPARQLWrapper* pour récupérer des données ;
- comment manipuler un graphe RDF en Python avec *rdflib* ;
- comment visualiser un graphe de connaissances RDF en Python avec *PyVis*.

Légende des pictogrammes utilisés dans ce *notebook* : 
|Picto.| Légende|
| ---- | ---- |
|🎬 | Action à réaliser : à vous de jouer ! | 
|💡| Suggestion d'action complémentaire |
|🏗️/▶️| Cellule à compléter / exécuter|
|⚠️| Avertissement |
|ℹ️| Information supplémentaire ou astuce |
|📚| Ressources : documentation, article, etc.|

<hr/>

## 🙤 Chapitre 4 : un graphe de connaissances sur les photos d'Eugène Atget

Le fichier `atget_photos.ttl` contient un petit graphe avec les URI des 4 216 photographies d'Atget recensées sur data.bnf.fr. Reste maintenant à récupérer les métadonnées de toutes ces photos !

### 1. Un graphe pour les rassembler toutes...

Commençons par charger le graphe `atget_photos` et vérifier qu'il contient bien le nombre de triplets attendus.

> 🏗️/▶️ Charger un graphe depuis un fichier très simple : il suffit de créer un nouvel objet `Graph` et d'utiliser la méthode `parse()`, qui accepte un nom de fichier et un paramètre optionnel `format=`.
> Complétez la cellule suivante pour charger le graphe d'index.

In [ ]:
from rdflib import Graph

FILENAME = "atget_photos.ttl"
FORMAT = "turtle"

atget_photos: Graph = ... #🏗️ Compléter

# Vérifications
assert isinstance(atget_photos, Graph), "Le résultat n'est pas un graphe RDFlib"
assert len(atget_photos) == 4216, "On attend un graphe de 4216 triplets"

In [ ]:
from utils import show_graph
show_graph(list(atget_photos)[:10]) # Affiche les 10 premiers triplets du graphe

### 2. ... mais en équipe !

Dans les prochaines étapes nous allons devoir à nouveau interroger le point d'accès SPARQL de la BnF pour récupérer toutes les métadonnées de 4 216 entités. Chaque groupe va faire la même opération, ce qui est lourd et inutilement redondant.

Mais nous pouvons nous partager le travail répartissant les photographies par équipe ! 😎

> ▶️ Assignez collectivement un numéro à chaque équipe constituée, de l'équipe 1 à l'équipe N. Puis **modifiez** et exécutez la cellule suivante pour découvrir **votre nom d'équipe** et récupérer **le sous-graphe qui vous est assigné** ! 😊

In [ ]:
NOMBRE_D_EQUIPES = 9 #🏗️ Changez par le nombre réel d'équipes constituées
VOTRE_NUMERO_D_EQUIPE = 4 #🏗️ Changez par le numéro de votre équipe (de 1 à NOMBRE_D_EQUIPE-1)

from utils import assign_to_team, your_team
from utils import TEAMS
team = TEAMS.get_team(VOTRE_NUMERO_D_EQUIPE)
graph_team = assign_to_team(atget_photos, NOMBRE_D_EQUIPES, VOTRE_NUMERO_D_EQUIPE)
your_team(graph_team, team)

### 3. Un graphe pour les ammener toutes ...

Nous voici avec une partie plus raisonnable du graphe, référençant quelques centaines de photographies.


Remarquez que les URIs des photographies dont on dispose dans le graphe d'équipe sont celles des **expressions** dans le modèle WEMI.
La prochaine étape va être d'utiliser le point d'accès SPARQL pour récupérer les URIs des **notices** et des **manifestations** correspondantes.

Nous avions vu dans le chapitre 1 que l'URI de la ressource "notice" est la racine des URIs expressions et manifestations, c'est à dire sans le fragment final `#Expression` ou `#about`.

> ▶️ Exécutez la cellule suivante pour créer, à partir du graphe `g`, la liste `expressions` de toutes les URIs des **expressions** de photographies contenues dans `g`.

In [ ]:
from rdflib import URIRef

expressions = []
pattern = (None, URIRef("http://id.loc.gov/vocabulary/relators/pht"), None)
for s, p, o in graph_team.triples(pattern):
    expressions.append(s)

expressions[:10]  # Affiche les 10 premières expressions recueillies

> 🏗️/▶️ Complétez la cellule suivante pour créer, à partir du graphe `g`, la liste `notices` de toutes les URIs **sans fragments**.
> Si vous le souhaitez, c'est le moment d'utiliser votre chatbot LLM préféré 🤖. N'oubliez pas : demandez **toujours** au 🤖 de commentez le code en détail, puis **lisez et comprenez** avant d'exécuter. 

In [ ]:
notices = []

for e in expressions:
    ... #🏗️ Compléter ici

# Vérifications
assert all(isinstance(n, URIRef) for n in notices), "Toutes les notices doivent être des URIRef"
assert len(set(notices)) == len(set(expressions)), "Le nombre de notices et d'expressions doit être identique"
print(notices[:10])  # Affiche les 10 premières notices recueillies

> 🏗️/▶️ Maintenant qu'on dispose des URIs des notices, on peut les utiliser pour générer les URIs des **manifestations** (en supposant qu'il y a une seule manifestation par notice !).

In [ ]:
manifestations = []

for n in notices:
    ... #🏗️ Compléter ici

# Vérifications
assert all(isinstance(m, URIRef) for m in manifestations), "Toutes les manifestations doivent être des URIRef"
assert len(set(manifestations)) == len(set(expressions)), "Le nombre de manifestations et d'expressions doit être identique"
print(manifestations[:10])  # Affiche les 10 premières manifestations recueillies

> 🏗️/▶️ Regroupez les URIs par groupes logiques de 3 : une notice, son expression, sa manifestation. Utilisez pour cela la méthode standard Python `zip` qui permet d'itérer sur plusieurs listes en parallèle. Un groupe = un tuple (notice, expression, manifestation)

In [ ]:
groups = []

#🏗️ Utiliser zip() ici pour itérer sur les listes expressions, notices et manifestations en parallèle, et remplir la liste groups de telle sorte 
# que chaque élément de groups soit un tuple (notice, expression, manifestation)
...

# Vérifications
assert all(isinstance(g, tuple) and len(g) == 3 for g in groups), "Chaque groupe doit être un tuple de 3 éléments"
assert all(all(isinstance(item, URIRef) for item in g) for g in groups), "Tous les éléments des groupes doivent être des URIRef"
print(groups[0])  # Affiche le premier groupe

> 🏗️/▶️ En vous aidant de ce que vous avez appris dans les chapitres précédents, interrogez le point d'accès SPARQL de data.bnf.fr pour chaque groupe,
> à l'aide d'une requête DESCRIBE contenant les 3 URIs représentant une photographie : sa notice, son expression, et sa manifestations.
> Vous devez choisir un format de retour qui permet de récupérer directement un graphe `rdflib`.

In [ ]:
# --- Construction des requêtes SPARQL `DESCRIBE <URI_NOTICE> <URI_EXPRESSION> <URI_MANIFESTATION>`
queries = []
for notice, expression, manifestation in groups:
    ... #🏗️ Compléter ici

# --- Initialisation du wrapper SPARQL ---
... #🏗️ Compléter ici

# --- Exécution des requêtes et collecte des graphes 
graphs = []
for query in queries:
    ... #🏗️ Compléter ici

In [ ]:
show_graph(graphs[0])  # Affiche le premier graphe récupéré

> ▶️ On peut sauvegarder chaque graphe de photographie dans un fichier séparé, que vous pouvez ensuite verser au **pot commun 📦** de l'atelier : **https://sdrive.cnrs.fr/s/N4gwbJNrR5CcWgb**
> Ce pot commun est un espace de stockage partagé, accessible à toutes les équipes participant à l'atelier et qui permet de centraliser les contributions de chacun pour construire collaborative le graphe complet des photographies d'Atget.


In [ ]:
# Créer le dossier d'export s'il n'existe pas
import os
export_dir = f"photographies"
os.makedirs(export_dir, exist_ok=True)

# Le nom est l'ID ARK de la ressource, que l'on peut extraire de l'URI de la notice (ark:/12148/<ID>)
graph_names = [n.split("ark:/12148/")[1] for n in notices]

# Exporter les graphes au format Turtle
for graph, name in zip(graphs, graph_names):
    graph.serialize(destination=os.path.join(export_dir, f"{name}.ttl"), format="turtle")

Vous pouvez maintenant **verser tous les fichiers .ttl au pot commun sur https://sdrive.cnrs.fr/s/N4gwbJNrR5CcWgb**  !

### 4. ...et toutes dans un même graphe les lier.

Il reste une ultime étape pour obtenir un graphe complet qui recense et décrit toutes les photographies de Eugène Atget recensées dans votre sous-graphe d'équipe : lier le graphe `graph_team` contenant l'index des photographies d'Atget avec **tous les graphes individuels** de métadonnées des photographies.
Et comme toujours, RDFlib rend les choses extrêmement simples puisqu'il est possible de fusionner deux graphes en écrivant simplement `g1 + g2`,  ou encore `g1 += g2`.

> 🏗️/▶️ Complétez le code suivant pour créer un graphe `graph_final`  qui combine le graphe d'index `graph_team` et tous les graphes de métadonnées des photographies.

In [ ]:
graph_final = Graph()

... #  🏗️ Compléter ici
    
print(f"Le graphe final contient {len(graph_final)} triplets.")

> ▶️ Enfin, on peut sauvegarder ce graphe servira pour la **PARTIE II** de l'atelier.

In [ ]:
graph_final.serialize(destination="graph_final.ttl", format="turtle")

> ▶️ Le graphe final contient trop de triplets pour être visualisé entièrement dans un notebook. On peut tout de même afficher le graphe de l'équipe avec les premiers graphes individuels.

ℹ️ Notez que les URIs servant d'identifiant uniques de ressources dans un graphe de connaissances, RDFlib a automatiquement fusionné les noeuds de même URI et a donc automatiquement lié nos graphes individuels à l'index initial 🪄✨

In [ ]:
# Affiche le graphe de l'équipe avec les 5 premiers graphes de photographies
show_graph(graph_team + graphs[0] + graphs[1] + graphs[2] + graphs[3] + graphs[4])

###  🏁 Fin du chapitre 4

Vous voilà arrivé.e.s à la fin de la **PARTIE I**, félicitations ! 🥳

Conservez précieusement le graphe final `final_graph.ttl`, vous en aurez besoin pour la **PARTIE II**.

<div style="text-align:center; font-size:xxx-large"> 🙤 FIN 🙦</div>